## 1. 데이터 불러오기

In [ ]:
import pandas as pd

# 학습 데이터 로딩
df_total = pd.read_csv('./data/df_total.csv')
df_test = pd.read_csv('./data/df_test.csv')

## 2. 데이터 전처리

In [ ]:
# 열 추가
df_total['기준일ID'] = pd.to_datetime(df_total['기준일ID'])

df_total['year']=df_total['기준일ID'].dt.year
df_total['month']=df_total['기준일ID'].dt.month
df_total['day']=df_total['기준일ID'].dt.day

In [ ]:
# 열 제거 및 이름 변경
df_total.rename(columns={'시간대구분': 'hour'}, inplace=True)
df_total = df_total.loc[:, ['year', 'month', 'day', 'hour', '총생활인구수']].copy()

In [ ]:
# 시프트를 사용해 target 설정
df_total['총생활인구수_M2'] = df_total['총생활인구수'].shift(-1416) # 2개월 후 동일 시간대의 총 생활인구수

In [ ]:
# 롤링을 사용해 feature 추가
# df_total['총생활인구수_MA_168'] = df_total['총생활인구수'].rolling(168).mean() # 기준일 포함 최근 7일간 이동 평균

In [ ]:
# 결측치 제거
cols = ['year', 'month', 'day', 'hour']

df_total.drop(cols, axis=1, inplace=True)
df_total.dropna(axis=0, inplace=True)

In [ ]:
train_x = df_total['총생활인구수']
train_y = df_total['총생활인구수_M2']

train_x.reset_index(drop=True, inplace=True)
train_y.reset_index(drop=True, inplace=True)

----

### 가. LinearRegression 부터 시작해봅시다.

#### [실습문제1] 머신러닝_1
* Train과 Test로 나눈 데이터를 기준으로 LinearRegression 모델링을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import *

In [ ]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    lr = LinearRegression()
    lr.fit(tx.reshape(-1, 1), ty)
    
    pred_lr = lr.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_lr, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_lr))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  1776.69081227391
R-squared Score:  0.8256857440269225

--- 1달 후의 생활인구수 예측 ---
RMSE:  2938.0656059078483
R-squared Score:  0.5312835971594729

--- 2달 후의 생활인구수 예측 ---
RMSE:  3194.7571725818657
R-squared Score:  0.44337793328743624

--- 1년 후의 생활인구수 예측 ---
RMSE:  3575.1304944341878
R-squared Score:  0.29243453783871876



#### [실습문제2] 머신러닝_2
* Train과 Test로 나눈 데이터를 기준으로 렌덤포레스트로 모델을 학습을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    rf = RandomForestRegressor()
    rf.fit(tx.reshape(-1, 1), ty)
    
    pred_rf = rf.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_rf, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_rf))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  2086.887958968078
R-squared Score:  0.7595042200749691

--- 1달 후의 생활인구수 예측 ---
RMSE:  3494.5976335812275
R-squared Score:  0.336896178767786

--- 2달 후의 생활인구수 예측 ---
RMSE:  3829.354844942175
R-squared Score:  0.20028372298524977

--- 1년 후의 생활인구수 예측 ---
RMSE:  4245.643265085746
R-squared Score:  0.0021395044513651262



#### [실습문제3] 머신러닝_3
* Train과 Test로 나눈 데이터를 기준으로 그라디언트부스팅으로 모델을 학습을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [ ]:
from xgboost import XGBRegressor

In [ ]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    xgb = XGBRegressor()
    xgb.fit(tx.reshape(-1, 1), ty)
    
    pred_xgb = xgb.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_xgb, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_xgb))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  1763.9992776997028
R-squared Score:  0.8281672266948278

--- 1달 후의 생활인구수 예측 ---
RMSE:  2921.1679095342
R-squared Score:  0.5366595504009777

--- 2달 후의 생활인구수 예측 ---
RMSE:  3189.933334670412
R-squared Score:  0.44505757738538154

--- 1년 후의 생활인구수 예측 ---
RMSE:  3547.7526173504066
R-squared Score:  0.3032299298115201



----